In [1]:
import sys
sys.path.append("../")

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import bayesflow as beef
from scipy.stats import beta, expon

from models import RandomWalkDDM
# from experiments import NeuralCoalMiningExperiment, BayesLoopCoalMiningExperiment
from priors import *

/Users/lukas/opt/miniconda3/envs/bfPower/lib/python3.10/site-packages/bayesflow/trainers.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
test = RandomWalkDDM()
test.generate(1)

In [3]:
test.test

2